In [ ]:
import nibabel as nib
import numpy as np
from scipy import stats
import mvlearn
from mvlearn import embed
import pandas as pd
import os
import re
import subprocess as sp
import glob
import hcp_utils as hcp


### Smooth time series using wb_command
def smooth_dtseries(in_dtseries, out_dtseries, kernel, L_surface, R_surface):
    sp.run(f'wb_command -cifti-smoothing -fwhm {in_dtseries} {kernel} {kernel} \
    COLUMN {out_dtseries} -left-surface {L_surface} -right-surface {R_surface}',shell=True)
    

### Load the time series of the L and R hemisphere 
def load_dtseries(dtseries_path):
    # load the file
    dtseries = nib.load(dtseries_path)
    # find the offset and number of vertices for each hemisphere
    L_model = [x for x in dtseries.header.get_index_map(1).brain_models if x.brain_structure=='CIFTI_STRUCTURE_CORTEX_LEFT']
    R_model = [x for x in dtseries.header.get_index_map(1).brain_models if x.brain_structure=='CIFTI_STRUCTURE_CORTEX_RIGHT']
    offset_count = [L_model[0].index_offset, L_model[0].index_count,
                    R_model[0].index_offset, R_model[0].index_count]
    # extract the cortical timeseries
    values = dtseries.get_fdata()
    values = values[0:,np.append(np.arange(offset_count[0],offset_count[0]+offset_count[1]),np.arange(offset_count[2],offset_count[2]+offset_count[3]))]
    
    return values


### Z-score and conatenation of dtseries of all subject folders in a directory
def concat_dtseries(paths_to_dtseries):
    if len(paths_to_dtseries)<2:
        print(f'At least 2 time series are needed: {len(paths_to_dtseries)} found')
    dtseries_lst = [load_dtseries(path) for path in paths_to_dtseries]
    dtseries_lst = [hcp.normalize(dtseries) for dtseries in dtseries_lst]
        
    return np.concatenate(dtseries_lst)


### Store gradients in cifti2 dscalar.nii
def mk_grad1_dscalar(grads, template_cifti, output_dir):
    # grads: array with dimensions gradients X vertices
    # template_cifti: any cifti2 file with a BrainModelAxis (I am using one of the dtseries.nii)
    # output_dir: path to output directory
    data = np.zeros([grads.shape[0],template_cifti.shape[1]])
    data[0:,0:grads.shape[1]] = grads

    map_labels = [f'Measure {i+1}' for i in range(grads.shape[0])]
    ax0 = nib.cifti2.cifti2_axes.ScalarAxis(map_labels)
    ax1 = nib.cifti2.cifti2_axes.from_index_mapping(template_cifti.header.get_index_map(1))
    nifti_hdr = template_cifti.nifti_header
    del template_cifti
    
    new_img = nib.Cifti2Image(data, header=[ax0, ax1],nifti_header=nifti_hdr)
    new_img.update_headers()

    new_img.to_filename(output_dir)
    

In [ ]:
### VARIABLES TO SET BEFORE RUNNING
# directory containing subdirectories named fter subject IDs that contain the timeseries and surface files
root_dir = "/home/fralberti/Data/HCP/"
# directory where all intermediate files and the final output will be saved
output_dir = "/home/fralberti/Data/Gradientile/"
# list of IDs of subjects to include in the analyses
subj_id = np.array(["100206","100307","100408","100610","101006",
                    "101107","101309","101410","101915","102008",
                    "102109","102311","102513","102614","102715"])

In [ ]:
### Smooth time series
print('Smoothing time series\nCurrent subject:')
for subj in subj_id[0:1]:
    dtseries_dirs = ['rfMRI_REST1_LR','rfMRI_REST1_RL','rfMRI_REST2_LR','rfMRI_REST2_RL']
    print(f'\t{subj}')
    for dir_tmp in dtseries_dirs:
        kernel = 2
        in_dtseries = f'{root_dir}{subj}/MNINonLinear/Results/{dir_tmp}/*_Atlas_MSMAll_hp2000_clean.dtseries.nii'
        out_dtseries = f'{root_dir}{subj}/MNINonLinear/Results/{dir_tmp}/{dir_tmp}_Atlas_MSMAll_hp2000_clean_smooth{kernel}.dtseries.nii'
        ### the surfaces should be reachable from the root!
        L_surface = f'{root_dir}{subj}/T1w/fsaverage_LR32k/{subj}.L.midthickness_MSMAll.32k_fs_LR.surf.gii'
        R_surface = f'{root_dir}{subj}/T1w/fsaverage_LR32k/{subj}.R.midthickness_MSMAll.32k_fs_LR.surf.gii'
        
        smooth_dtseries(in_dtseries, out_dtseries, kernel, L_surface, R_surface)
print('Done!')

In [ ]:
### Normalize and conatenate time series
print('Concatenating smoothed time series\nCurrent subject:')
for subj in subj_id:
    print(f'\t{subj}')
    paths_to_dtseries = glob.glob(f'{root_dir}{subj}/MNINonLinear/Results/*/*REST1*_smooth6.dtseries.nii')
    dtseries_out = concat_dtseries(paths_to_dtseries)
    np.save(f'{output_dir}/{subj}.rfMRI_REST1_all.npy',dtseries_out)
    del dtseries_out
print('Done!')

In [ ]:
### Compute GCCA
batch_size = 3
batch_num = int(len(subj_id)/batch_size)
batches = [np.arange(x,x+batch_size) for x in np.arange(0,len(subj_id),batch_size)]
template_cifti = nib.load(f'{root_dir}HCP_S1200_GroupAvg_v1/S1200.All.midthickness_MSMAll_va.32k_fs_LR.dscalar.nii')

print("Running GCCA\nCurrent batch:")
for batch_tmp in batches:
    print(f'\t{subj_id[batch_tmp]}')
    
    dtseries_batch = [np.array(np.load(f'{output_dir}{subj}.rfMRI_REST1_all.npy')).T for subj in subj_id[batch_tmp]]
    res = embed.GCCA(n_components=4)
    res = res.fit_transform(dtseries_batch)
    del dtseries_batch

    for i,subj in enumerate(subj_id[batch_tmp]):
        grads = res[i,:,:].T
        mk_grad1_dscalar(grads, template_cifti, f'{output_dir}{subj}.REST1_gcca_smooth6.dscalar.nii')
    del res

print("Done!")

In [ ]:
### Invert gradients to allign across subject (if needed)

# subj_to_invert = subj_id
# for subj in subj_to_invert:
#     grads = nib.load(f'{output_dir}{subj}.REST1_gcca_smooth6.dscalar.nii')
#     grads_inv = -1*grads.get_fdata()
#     mk_grad1_dscalar(grads_inv, grads, f'{output_dir}{subj}.REST1_gcca_smooth6.dscalar.nii')